In [1]:
import operator as op
import src.ea.ea_lib as ea
import src.ea.problem_data as pb
import src.ea.utilities as ut
import random as rd
import time
import numpy as np

In [2]:
#parameters
rd.seed(4)
generations = 20
population_size = 200
tournament_size = 5
mutation_percentage = 9
problem_name = "spect"
levels_back = 2
n_rows = 4
n_columns = 9
allow_input_to_output = False
inputs_available_to_all_columns = False
functions = [op.add,op.sub,op.mul,ut.safe_divide_one]
objectives = [
    ea.Objective(name="acc0", to_max = True, best=1, worst=0),
    ea.Objective(name="acc1", to_max = True, best=1, worst=0)
    ]
#The generation of creation of each individual is to be taken into account in cgp: the newest individuals are preferred over the oldest
generation_objective = ea.Objective(name="generation", to_max = True)

In [3]:
#Definition
dataset = pb.Dataset()
dataset.load_problem(name = problem_name)
nsgaii_objectives = ea.get_nsgaii_objectives()
cgp = ea.CGP_Representation(
            dataset.x_train.shape[1]
            ,1
            ,levels_back
            ,n_rows 
            ,n_columns
            ,allow_input_to_output
            ,inputs_available_to_all_columns
            ,*functions)

#Extraction of useful variables
labels = list(set(dataset.y_train))
print(labels)

[0, 1]


In [4]:
def evaluate_ind(ind):
    outputs = {}
    for i,data_row in enumerate(dataset.x_train):
        output_dict = ind.representation.evaluate(data_row = data_row)
        #Extracting the 0th index output gene
        output = output_dict[0]
        #The raw output needs to be transformed
        #transformed_output = ut.custom_round(output)
        transformed_output = ut.threshold_map(value = output,threshold = 0.5, output_up = labels[1], output_down = labels[0])
        outputs[i] = transformed_output
    #Each objective has its own evaluation method
    acc0 = ut.accuracy_in_label(y = dataset.y_train, y_output = outputs, label = 0)
    acc1 = ut.accuracy_in_label(y = dataset.y_train, y_output = outputs, label = 1)
    ind.update_evaluation(objective = objectives[0], value = acc0)
    ind.update_evaluation(objective = objectives[1], value = acc1)
    ind.update_semantics_all(semantics_all = outputs)

def evaluate_population(population):
    for ind in population:
        evaluate_ind(ind)

def print_logs(population):
    evals0 = []
    evals1 = []
    actives = []
    gens = []
    for i,ind in enumerate(population):
        evals0.append(ind.evaluations[objectives[0].name])
        evals1.append(ind.evaluations[objectives[1].name])
        actives.append(len(ind.representation.active_genotype))
        gens.append(ind.evaluations["generation"])
    print("Best", max([evals0[i]+evals1[i] for i in range(len(population))]))
    print("Mean acc0", np.mean(evals0))
    print("Mean acc1", np.mean(evals1))
    print("Mean gen", np.mean(gens))
    print("Mean active nodes", np.mean(actives))
    print("Skips", len([i for i in population if i.representation.evaluation_skipped]))
    print(" ")

In [5]:
#Initial generation
generation = 0

#Random initial population. Specific initial conditions for the population can be specified here
graphs = [cgp.create_random(seed = rd.random()) for _ in range(population_size)]

#create instances of Individual to be grouped in the population
parent_population = [ea.Individual(r) for r in graphs]

#Evaluate and sort the population according to non-domination
evaluate_population(population = parent_population)
sorted_nsga2_population = ea.fast_nondominated_sort(population = parent_population, conflicting_objectives = objectives, nsgaii_objectives = nsgaii_objectives)

#Create the offsprings of the initial generation
population = parent_population
for i in range(population_size):

    #Binary tournament selection is used in the initial generation only according to NSGA-II. The offspring is evaluated and added to the population
    parent_index = ea.tournament_selection_index(population_size = population_size, tournament_size = 2)
    parent = sorted_nsga2_population[parent_index]
    new_graph, active_altered = cgp.point_mutation(graph = parent.representation, percentage = mutation_percentage)
    offspring = ea.Individual(representation = new_graph)

    #If the active graph was not altered, the individual does not need to be evaluated again:
    if active_altered:
        evaluate_ind(offspring)
    else:
        offspring.representation.evaluation_skipped = True
        for objective in objectives:
            offspring.update_evaluation(objective = objective, value = parent.evaluations[objective.name])
        offspring.update_semantics_all(semantics_all = parent.semantics_all)
    population.append(offspring)

for ind in population:
    ind.update_evaluation(objective = generation_objective, value = generation)

In [6]:

new_pop_size = len(population)
for _ in range(generations):
    generation += 1
    start_t = time.time()

    #Sorting the population. The generation of creation is the latest tiebreak (suggested in CGP), so fast_nondominated_sort cannot be used as it is
    front_objective = nsgaii_objectives[0]
    cd_objective = nsgaii_objectives[1]
    ea.set_ranks(population = population, conflicting_objectives = objectives, front_objective = front_objective)
    ea.set_crowding_distances_by_front(population = population, conflicting_objectives = objectives, front_objective = front_objective, cd_objective = cd_objective)
    sorted_population = ea.sort_population(population = population, objectives = [front_objective, cd_objective, generation_objective])

    #Elitism
    parent_population = sorted_population[:population_size]

    #Offspring generation
    offspring_population = []
    for i in range(population_size):
        parent_index = ea.tournament_selection_index(population_size = len(parent_population), tournament_size = tournament_size)
        parent = parent_population[parent_index]
        new_graph, active_altered = cgp.point_mutation(graph = parent.representation, percentage = mutation_percentage)
        offspring = ea.Individual(representation = new_graph)

        #If the active graph was not altered, the individual does not need to be evaluated again:
        if active_altered:
            evaluate_ind(offspring)
        else:
            offspring.representation.evaluation_skipped = True
            for objective in objectives:
                offspring.update_evaluation(objective = objective, value = parent.evaluations[objective.name])
            offspring.update_semantics_all(semantics_all = parent.semantics_all)
        offspring_population.append(offspring)

    #Update the generation of creation of the offsprings
    for offspring in offspring_population:
        offspring.update_evaluation(objective = generation_objective, value = generation)

    #Formation of the population for the next gen
    population = offspring_population + parent_population

    print("Gen time: ",str(time.time()-start_t))
    print_logs(population)


Gen time:  1.846036434173584
Best 0.7265917602996255
Mean acc0 0.32327715355805237
Mean acc1 0.22602059925093634
Mean gen 0.5
Mean active nodes 6.92
Skips 112
 
Gen time:  1.7428865432739258
Best 0.7265917602996255
Mean acc0 0.3170692883895131
Mean acc1 0.24820224719101128
Mean gen 1.2875
Mean active nodes 5.4475
Skips 190
 
Gen time:  1.6595211029052734
Best 0.7565543071161049
Mean acc0 0.387312734082397
Mean acc1 0.22135767790262173
Mean gen 2.2125
Mean active nodes 4.305
Skips 246
 
Gen time:  1.617656946182251
Best 0.7565543071161049
Mean acc0 0.3172378277153558
Mean acc1 0.2974250936329588
Mean gen 3.1075
Mean active nodes 3.805
Skips 264
 
Gen time:  1.572765827178955
Best 0.7565543071161049
Mean acc0 0.3118726591760299
Mean acc1 0.3027902621722846
Mean gen 4.235
Mean active nodes 3.1425
Skips 280
 
Gen time:  1.725402593612671
Best 0.7565543071161049
Mean acc0 0.3009831460674157
Mean acc1 0.3039325842696629
Mean gen 5.2325
Mean active nodes 4.0525
Skips 254
 
Gen time:  1.691457

In [7]:
ea.plot_pareto(population = parent_population, conflicting_objectives = objectives, front_objective = nsgaii_objectives[0])

ValueError: x and y must be the same size

In [ ]:
x = [i.evaluations[objectives[0].name] for i in parent.population]
y = [i.evaluations[objectives[1].name] for i in parent.population]
    

plt.scatter(x, y, alpha=0.5)
plt.show()